# Section 5: Sampler Primitive - Practice Questions

**Exam Weight**: 12% (~8 questions) | **Difficulty**: Medium-High | **Must Master**: ✅✅✅

---

## 🎯 Key Traps to Watch For:

| Trap | Wrong Assumption | Correct Understanding |
|------|------------------|----------------------|
| PUB format | `[circuit]` | `[(circuit,)]` - TUPLE with trailing comma! |
| Sampler circuits | No measurements needed | MUST have `measure()` or `measure_all()` |
| Result access | `result.data` | `result[0].data` - index with [0] first! |
| get_counts() | On result directly | `result[0].data.meas.get_counts()` |
| Shots | Set in circuit | Set in PUB or primitive options |
| Multiple circuits | One result object | `result[i]` for each circuit |

> 📖 See section_5_sampler/README.md for full concepts

---

## 📚 Topics Covered (from Section Notebooks):

### Sampler Primitive (`sampler_primitive.ipynb`)

#### StatevectorSampler (Local Simulation)
- **Creation**: `from qiskit.primitives import StatevectorSampler`
- **Running**: `sampler.run([(circuit,)])` - PUB format!
- **Ideal results**: No noise, exact probabilities

#### SamplerV2 (Runtime)
- **Creation**: `from qiskit_ibm_runtime import SamplerV2`
- **PUB format**: `(circuit, parameter_values, shots)`
- **Options**: `SamplerV2(backend, options=options)`

#### Result Extraction
- **Full path**: `result[0].data.meas.get_counts()`
- **BitArray methods**: `.get_counts()`, `.get_bitstrings()`, `.num_shots`
- **Register access**: `result[0].data.<register_name>`

#### Advanced Features
- **Twirling**: `enable_gates`, `enable_measure`
- **Dynamical Decoupling**: `dd.enable`, `dd.sequence_type`
- **Multiple PUBs**: Run batch of circuits

In [ ]:
# Setup - Run this first!
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.primitives import StatevectorSampler
from qiskit.circuit import Parameter
import numpy as np
%matplotlib inline
print("✅ Setup complete!")

---
## Part 1: StatevectorSampler Basics

⚠️ **EXAM CRITICAL**: The Sampler API changed in Qiskit 1.0!

| Old API (pre-1.0) | New API (1.0+) |
|-------------------|----------------|
| `Sampler().run(circuit)` | `StatevectorSampler().run([(circuit,)])` |
| `result.quasi_dists[0]` | `result[0].data.meas.get_counts()` |

### Q1: Basic Sampler usage

In [ ]:
# Your solution: Create Bell state, measure, run with StatevectorSampler

In [ ]:
# Solution Q1
# Create circuit with measurement
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.measure_all()  # REQUIRED for Sampler!

# Create sampler and run
sampler = StatevectorSampler()

# PUB format: list of tuples, each tuple is (circuit,) or (circuit, params)
job = sampler.run([(qc,)])  # Note: tuple (qc,) not just qc!
result = job.result()

# Extract counts
counts = result[0].data.meas.get_counts()
print(f"Counts: {counts}")

### Q2: PUB format deep dive

In [ ]:
# Your solution: Show correct and incorrect PUB formats

In [ ]:
# Solution Q2
qc = QuantumCircuit(1)
qc.h(0)
qc.measure_all()

sampler = StatevectorSampler()

# ✅ CORRECT: List of tuples
job = sampler.run([(qc,)])
result = job.result()
print(f"Correct format: {result[0].data.meas.get_counts()}")

# Also correct: multiple PUBs
job2 = sampler.run([(qc,), (qc,)])
result2 = job2.result()
print(f"Multiple PUBs: {len(result2)} results")

# ❌ WRONG: sampler.run(qc) - no tuple
# ❌ WRONG: sampler.run([qc]) - no tuple inside list

---
## Part 2: Running with Shots

### Q3: Specify number of shots

In [ ]:
# Your solution: Run with different shot counts and compare results

In [ ]:
# Solution Q3
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.measure_all()

sampler = StatevectorSampler()

# Default shots (perfect simulation)
result_default = sampler.run([(qc,)]).result()
print(f"Default (perfect): {result_default[0].data.meas.get_counts()}")

# With specific shots (introduces sampling noise)
result_100 = sampler.run([(qc,)], shots=100).result()
print(f"100 shots: {result_100[0].data.meas.get_counts()}")

result_1000 = sampler.run([(qc,)], shots=1000).result()
print(f"1000 shots: {result_1000[0].data.meas.get_counts()}")

---
## Part 3: Parameterized Circuits

### Q4: Run parameterized circuit

In [ ]:
# Your solution: Create parameterized circuit, bind values in PUB

In [ ]:
# Solution Q4
theta = Parameter('θ')

qc = QuantumCircuit(1)
qc.ry(theta, 0)
qc.measure_all()

sampler = StatevectorSampler()

# PUB with parameter values: (circuit, parameter_values)
# Test different angles
angles = [0, np.pi/2, np.pi]
pubs = [(qc, [angle]) for angle in angles]  # Each PUB: (circuit, [params])
print(f"PUBs: {pubs}")

result = sampler.run(pubs, shots=1000).result()
qc.draw('mpl')
for i, angle in enumerate(angles):
    counts = result[i].data.meas.get_counts()
    print(f"θ={angle:.2f}: {counts}")

### Q5: Multiple parameter values at once

In [ ]:
# Your solution: Run circuit with array of parameter values

In [ ]:
# Solution Q5
theta = Parameter('θ')
qc = QuantumCircuit(1)
qc.ry(theta, 0)
qc.measure_all()

sampler = StatevectorSampler()

# Broadcasting: 2D array of parameter values
# Shape: (num_parameter_sets, num_parameters)
param_values = np.array([[0], [np.pi/4], [np.pi/2], [3*np.pi/4], [np.pi]])

# Single PUB with multiple parameter sets
result = sampler.run([(qc, param_values)], shots = 10000).result()

# Result contains all outcomes
pub_result = result[0]
print(f"Number of parameter sets: {len(param_values)}")
print(f"\nCounts for each parameter set:")
for i, pv in enumerate(param_values):
    # Access via BitArray indexing
    counts = pub_result.data.meas.get_counts()
    print(f"  θ={pv[0]:.2f}: {counts}")

---
## Part 4: Result Extraction

| Path | Returns |
|------|--------|
| `result[pub_idx]` | PubResult for one PUB |
| `result[i].data.meas` | BitArray of measurements |
| `result[i].data.meas.get_counts()` | Counts dict |

### Q6: Navigate result structure

In [ ]:
# Your solution: Explore all parts of the result object

In [ ]:
# Solution Q6
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.measure_all()

result = StatevectorSampler().run([(qc,)], shots=100).result()

# Navigate the hierarchy
print("Result structure:")
print(f"  Type: {type(result)}")
print(f"  Number of PUBs: {len(result)}")

pub_result = result[0]
print(f"\nPubResult:")
print(f"  Type: {type(pub_result)}")
print(f"  Data registers: {pub_result.data}")

bit_array = pub_result.data.meas
print(f"\nBitArray:")
print(f"  Type: {type(bit_array)}")
print(f"  Num bits: {bit_array.num_bits}")
print(f"  Num shots: {bit_array.num_shots}")

counts = bit_array.get_counts()
print(f"\nCounts: {counts}")

### Q7: Multiple PUBs in one job

In [ ]:
# Your solution: Run multiple circuits and extract each result

In [ ]:
# Solution Q7
# Create different circuits
qc1 = QuantumCircuit(1)
qc1.h(0)
qc1.measure_all()

qc2 = QuantumCircuit(2)
qc2.h(0)
qc2.cx(0, 1)
qc2.measure_all()

qc3 = QuantumCircuit(3)
qc3.h([0, 1, 2])
qc3.measure_all()

# Run all as separate PUBs
sampler = StatevectorSampler()
result = sampler.run([(qc1,), (qc2,), (qc3,)], shots=100).result()

# Extract each result
for i in range(len(result)):
    counts = result[i].data.meas.get_counts()
    print(f"Circuit {i+1}: {counts}")

---
## Part 5: Named Classical Registers

### Q8: Access named registers

In [ ]:
# Your solution: Create circuit with named classical register, access results

In [ ]:
# Solution Q8
from qiskit import ClassicalRegister, QuantumRegister

qr = QuantumRegister(2, 'q')
cr = ClassicalRegister(2, 'result')  # Named register!
qc = QuantumCircuit(qr, cr)
qc.h(0)
qc.cx(0, 1)
qc.measure(qr, cr)

result = StatevectorSampler().run([(qc,)], shots=100).result()

# Access by register name
pub_result = result[0]
print(f"Available registers: {dir(pub_result.data)}")

# Access by name 'result'
counts = pub_result.data.result.get_counts()
print(f"\nCounts from 'result' register: {counts}")

### Q9: measure_all() vs explicit measure

In [ ]:
# Your solution: Compare register names for measure_all vs measure

In [ ]:
# Solution Q9
# Circuit 1: measure_all() creates register named 'meas'
qc1 = QuantumCircuit(2)
qc1.h(0)
qc1.measure_all()

result1 = StatevectorSampler().run([(qc1,)]).result()
print("measure_all() -> register 'meas':")
print(f"  {result1[0].data.meas.get_counts()}")

# Circuit 2: explicit measure to named register
qc2 = QuantumCircuit(2, 2)
qc2.h(0)
qc2.measure([0, 1], [0, 1])

result2 = StatevectorSampler().run([(qc2,)]).result()
print("\nExplicit measure -> register 'c':")
print(f"  {result2[0].data.c.get_counts()}")

---
## Part 6: Common Exam Patterns

### Q10: Complete workflow

In [ ]:
# Your solution: Full workflow - create, measure, sample, extract

In [ ]:
# Solution Q10: Complete exam-style workflow

# Step 1: Create circuit
qc = QuantumCircuit(3)

# Step 2: Apply gates (GHZ state)
qc.h(0)
qc.cx(0, 1)
qc.cx(1, 2)

# Step 3: Add measurement (REQUIRED for Sampler!)
qc.measure_all()

# Step 4: Create sampler
sampler = StatevectorSampler()

# Step 5: Run with PUB format
job = sampler.run([(qc,)], shots=1000)

# Step 6: Get result
result = job.result()

# Step 7: Extract counts
counts = result[0].data.meas.get_counts()

print("GHZ state measurement:")
print(f"Counts: {counts}")
print(f"\nExpected: Only '000' and '111' (entangled state)")

### Q11: Sampler vs Estimator Decision (EXAM QUESTION!)

| Need | Primitive | Why |
|------|-----------|-----|
| Measurement counts | **Sampler** | Get bitstring samples |
| Expectation values ⟨O⟩ | **Estimator** | Calculate energy |
| Grover output | **Sampler** | Find marked state |
| VQE energy | **Estimator** | Minimize Hamiltonian |

---
## ✅ Section 5 Checklist

**StatevectorSampler**:
- [ ] `StatevectorSampler()` - create sampler
- [ ] `sampler.run(pubs, shots=N)` - run job

**PUB Format** (CRITICAL!):
- [ ] `[(circuit,)]` - single circuit, no params
- [ ] `[(circuit, [params])]` - with parameter values
- [ ] `[(qc1,), (qc2,)]` - multiple PUBs

**Result Extraction**:
- [ ] `result[pub_idx]` - get PubResult
- [ ] `result[i].data.meas` - BitArray (for measure_all)
- [ ] `result[i].data.<register_name>` - named register
- [ ] `.get_counts()` - get counts dict

**Common Errors**:
- [ ] ❌ `sampler.run(qc)` - must be `[(qc,)]`
- [ ] ❌ Forgetting `measure_all()` on circuit
- [ ] ❌ Using old API `.quasi_dists[0]`